In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
# Add the fourier_learning_ibm package to the path
# This is necessary only when running the notebook in the docker container
import sys, pprint

sys.path.append("/home/jovyan/fourier_learning_ibm/")
pprint.pprint(sys.path)

# This is necessary only when running the notebook in the docker container
%cd fourier_learning_ibm/

In [ ]:
import numpy as np
from numpy.linalg import qr
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import time
import networkx as nx
from heisenberg import (
    HeisenbergModel,
    get_graph,
    get_positions,
)
from setup import setup_backend
from heisenberg import HeisenbergModel, get_graph, get_positions
from qiskit import transpile
from qiskit.quantum_info.operators import Operator
from qiskit.quantum_info import (
    Statevector,
    state_fidelity,
)
import json

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# state vector simulator
backend_qpu, backend_sim_noiseless, backend_sim_noisy = setup_backend(
    qpu_name="ibm_marrakesh"
)

In [ ]:
# # MPS simulator
# backend_qpu, backend_sim_noiseless, backend_sim_noisy = setup_backend(
#     qpu_name="ibm_marrakesh", method="matrix_product_state"
# )

In [ ]:
# Config number of qubits
n_qubits = 4

## 積公式エラーの計算　(やらなくても良い)

実機ノイズがない状態で、積公式によるエラーを調べる。

In [ ]:
max_n_step = 500
times = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] + list(np.arange(10, 200, 5))
results = []

rng = np.random.default_rng(42)
Js = rng.uniform(-1, 1, size=n_qubits - 1)
G = get_graph(n_qubits, Js)
heisenberg = HeisenbergModel(n_qubits, G)

for t in times:
    # initial state
    state_init = Statevector.from_label("0" * n_qubits)

    # Exact simulation (Non-Trotter)
    state_exact, U_exact = heisenberg.exact_simulation(t, phase=0)

    # Trotter simulation
    for n_step in range(0, round(max_n_step / 50) * 50 + 1, 50):  # 50 刻みで計算する
        if n_step == 0:
            n_step = 1

        circuit = heisenberg.get_circuit(t, n_step, phase=0)
        circuit.remove_final_measurements()

        print("preparing U_trotter...")
        U_trotter = Operator.from_circuit(circuit)

        print("processing...")
        state_trotter = state_init.evolve(U_trotter)

        print("calculating fidelity...")
        sta_fidelity = state_fidelity(state_exact, state_trotter)
        print(f"t: {t:.3f}, n_step: {n_step}, fidelity: {sta_fidelity:.3f}")

        results.append(
            {
                "t": t,
                "n_step": n_step,
                "sta_fidelity": sta_fidelity,
            }
        )

In [ ]:
df = pd.DataFrame(results)
# t と n_step ごとに sta_fidelity をグループ化
df_grouped = df.groupby(["t", "n_step"]).agg({"sta_fidelity": "mean"})
df_grouped = df_grouped.reset_index()
display(df_grouped)

# plot
plt.figure(figsize=(10, 5))
for n_step in df_grouped["n_step"].unique():
    df_tmp = df_grouped[df_grouped["n_step"] == n_step]
    plt.plot(df_tmp["t"], df_tmp["sta_fidelity"], label=f"n_step={n_step}", marker="o")
# random_pro_fidelity = 1 / (2**n_qubits)
# plt.axhline(y=random_pro_fidelity, color="red", linestyle="dashed", label="1/(2**n)")
plt.xlabel("Time")
plt.ylabel("State Fidelity")
plt.legend(bbox_to_anchor=(0, -0.1), loc="upper left")
plt.grid()
plt.title(f"Stete Fidelity vs Time ({n_qubits}Q)")
# plt.savefig(f"results/get_param/{n_qubits}Q/fidelity_vs_time.pdf")
plt.show()

In [ ]:
max_times = {}
threshold = 0.98

# 各 n_step ごとに、threshold を超える最大の time を取得
for n_step in df_grouped["n_step"].unique():
    df_tmp = df_grouped[df_grouped["n_step"] == n_step]
    max_time = df_tmp[df_tmp["sta_fidelity"] > threshold]["t"].max()
    max_times[n_step] = max_time

print(max_times)